In [ ]:
import os
import pickle
import numpy as np

from PIL import Image
from tqdm import tqdm
from typing import List
from torchvision import transforms

In [ ]:
def crop_and_split(image_path: str, n: int) -> List[np.ndarray]:
    """
    Crops the center nxn region of an image and splits it into 9 smaller (n/3)x(n/3) images.

    Parameters
    ----------
    image_path : str
        The file path to the image.
    n : int
        The center image size to crop

    Returns
    -------
    List[np.ndarray]
        A list of 9 patches, each of size 50x50, extracted from the center cropped image.
    """

    transform = transforms.Compose([
        transforms.Grayscale(),  
        transforms.CenterCrop(n) 
    ])
    
    img = Image.open(image_path)
    img = transform(img)

    img_np = np.array(img)
    
    patches = []
    for i in range(3):
        for j in range(3):
            patch = img_np[i*int(n/3):(i+1)*int(n/3), j*int(n/3):(j+1)*int(n/3)]
            patches.append(patch)
    
    return patches

In [ ]:
# Path to ImageNet Large Scale Visual Recognizition Challenge 2012 Validation Set (ILSVRC 2012)
folder_path = '/Users/wizard/Downloads/ILSVRC2012_img_val'

# Matrix To Store All Entries
img_mtrx = np.zeros((450000, 2500), dtype = np.float32)

curr_idx = 0
for i in tqdm(range(1, 50001), desc = 'Processing'):
    path = os.path.join(folder_path, f'ILSVRC2012_val_{i:08d}.JPEG')

    patches = crop_and_split(path, n = 150)

    for patch in patches:
        img_mtrx[curr_idx] = patch.flatten()
        curr_idx += 1
    

In [ ]:
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/ILSVRC2012_50x50.pkl', 'wb') as f:
    pickle.dump(img_mtrx, f)